# Setup

In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai

PROJECT_ID = "kaggle-on-gcp"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from IPython.display import Markdown, display
from vertexai.language_models import CodeChatModel

In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
# code_chat_model = CodeChatModel.from_pretrained("codechat-bison-32k")

# Code Executor Raw

In [ ]:
from subprocess import run
def execute_code(code):
    try:
        exec(code)
        result = "Success"
    except Exception as e:
        result = f"Error: {str(e)}"
    return result

def code_executer(chat_model, problem_statement = "", debug=True):
  run_flag = False
  code_chat = chat_model.start_chat()
  step_1 = (code_chat.send_message(
      f"Please help write a python function {problem_statement}",
  ))
  step_2 = code_chat.send_message(
      """
            Modify the provided Python function so that it can be executed in the Python kernel.
            Ensure the function includes necessary function calls and test cases.
            - Do not use the unittest library or create Python classes.
            - Make sure to add all the import statements at the top of the function.
            - Do not add "if __name__ == "__main__": to call the function. Just do simple function calling.
            - If the test passes, print "test pass".
            - If the test fails, print "test fail".
        """,
  )
  if debug:
    print("1st generation: \n", step_2.text )
  #execute the code that is prodcued
  code_to_execute = step_2.text.split("```")[1].split('python')[1]
  if debug:
    print("1st run code: \n", code_to_execute )
  execution_result = execute_code(code_to_execute)
  if debug:
    print("1st run results: \n", execution_result)
  if execution_result == "Success":
      print("The generated code is running without any error, However it may or maynot pass some test cases: \n\n")
      print(code_to_execute)
  else:
      print("The generated code isn't working, asking the model to fix it")
      run_flag = True
      i = 0
      while(run_flag and i <4):
          print("Fixing Try...... ", i+1)
          step_3 = code_chat.send_message(
              f"""The code throws an error: {execution_result}.
                - Re-write the code so that it can run on the Python kernel.
                - If the error is due to library installation \
                    add lines that can install the libraries with '!' like '!pip install pandas`.
                """
          )
          code_to_execute = step_3.text.split("```")[1].split('python')[1]
          execution_result = execute_code(code_to_execute)
          if execution_result == "Success":
              print("The generated code is running without any error, after fixing the issues: \n\n")
              print(code_to_execute)
              run_flag = False
          else:
            run_flag = True
            i += 1
            if i == 3:
              print("""There's something wrong with the code, model is unable to fix in 4 iterations.
                    You can run with debug=True and see for yourself.
                    Maybe re-write the prompt and ask again. Thank You!
                    """)


In [ ]:
problem_statement = """to print the first non-repeated character from a string.
                      take example of 'Mississippi' and 'hello' as example.
                      """
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = problem_statement,debug=False)

Test case 1 failed
Test case 2 failed
The generated code is running without any error, However it may or maynot pass some test cases: 



import collections

def first_non_repeating_char(str1):
    char_order = []
    char_count = collections.Counter(str1)

    # Find the first character that does not repeat.
    for c in char_order:
        if char_count[c] == 1:
            return c

    # Return -1 if no non-repeating character is found.
    return -1

# Test case 1
test_str1 = 'Mississippi'
expected_output1 = 's'
output1 = first_non_repeating_char(test_str1)
if output1 == expected_output1:
    print("Test case 1 passed")
else:
    print("Test case 1 failed")

# Test case 2
test_str2 = 'hello'
expected_output2 = 'h'
output2 = first_non_repeating_char(test_str2)
if output2 == expected_output2:
    print("Test case 2 passed")
else:
    print("Test case 2 failed")



In [ ]:
problem_statement = """
that defines a class called Restaurant with three attributes: name, address, and cuisine.
Then, write a function called find_matching_restaurants() that takes a list of Restaurant objects and a search query \
as input and returns a list of Restaurant objects that match the search query.
The search query can be a substring of the restaurant name, address, or cuisine type.
here's an example of using the following function:
restaurants = [
              Restaurant("The Grill", "123 Main Street", cuisine="American"),
              Restaurant("Thai Paradise", "456 Elm Street", cuisine="Thai"),
              Restaurant("Little Italy", "789 Pine Street", cuisine="Italian"),
              ]
search_query = "Italian"
output: 'Little Italy'
"""
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = problem_statement,debug=False)

Test passed
The generated code is running without any error, However it may still not pass some test cases: 



class Restaurant:
  def __init__(self, name, address, cuisine):
    self.name = name
    self.address = address
    self.cuisine = cuisine

def find_matching_restaurants(restaurants, search_query):
  matching_restaurants = []
  for restaurant in restaurants:
    if search_query.lower() in restaurant.name.lower() or search_query.lower() in restaurant.address.lower() or search_query.lower() in restaurant.cuisine.lower():
      matching_restaurants.append(restaurant)
  return matching_restaurants

restaurants = [
              Restaurant("The Grill", "123 Main Street", cuisine="American"),
              Restaurant("Thai Paradise", "456 Elm Street", cuisine="Thai"),
              Restaurant("Little Italy", "789 Pine Street", cuisine="Italian"),
              ]
search_query = "Italian"
result = find_matching_restaurants(restaurants, search_query)

if result[0].name == "Little Ital

In [ ]:
problem = """
          that can extract ipv4 addresses from the each line in the log file.
                        input:
                        03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0
                        output:
                        129.1.1.1
                        \n\n
          """
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = problem,debug=False)

Test passed.
The generated code is running without any error, However it may still not pass some test cases: 



import re

def extract_ipv4_addresses(log_line):
  """Extracts IPv4 addresses from a log line.

  Args:
    log_line: A string containing a log line.

  Returns:
    A list of IPv4 addresses.
  """

  ipv4_addresses = []
  for match in re.finditer(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b", log_line):
    ipv4_addresses.append(match.group())
  return ipv4_addresses


# Test the function.

log_line = "03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0"
expected_ipv4_addresses = ["129.1.1.1"]
ipv4_addresses = extract_ipv4_addresses(log_line)
if ipv4_addresses == expected_ipv4_addresses:
  print("Test passed.")
else:
  print("Test failed.")



In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = "check if two strings are a rotation of each other",debug=False)

Test passed
Test passed
The generated code is running without any error, However it may still not pass some test cases: 



def are_rotations(string1, string2):
    size1 = len(string1)
    size2 = len(string2)

    # Check if sizes of two strings are same
    if size1 != size2:
        return False

    # Create a temp string with value str1.str1
    temp = string1 + string1

    # Now check if str2 is a substring of temp
    # string.find() returns -1 if str2 is not present
    # in temp
    if (temp.find(string2) == -1):
        return False

    return True


# Test cases
string1 = "ABCD"
string2 = "CDAB"
if (are_rotations(string1, string2)):
    print("Test passed")
else:
    print("Test failed")

string1 = "AACD"
string2 = "ACDA"
if (are_rotations(string1, string2)):
    print("Test passed")
else:
    print("Test failed")



In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = "count a number of vowels and consonants in a given string",
                 debug=False)

Test pass
Test fail
Test pass
Test fail
The generated code is running without any error, However it may still not pass some test cases: 



def count_vowels_and_consonants(string):
  vowels = 'aeiouAEIOU'
  consonants = 'bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ'
  vowel_count = 0
  consonant_count = 0
  for letter in string:
    if letter in vowels:
      vowel_count += 1
    elif letter in consonants:
      consonant_count += 1
  return vowel_count, consonant_count

# Test cases
test_cases = [
  ('Hello', 2, 3),
  ('The quick brown fox jumps over the lazy dog', 10, 18),
  ('AEIOU', 5, 0),
  ('BCDFGHJKLMNPQRSTVWXYZ', 0, 26)
]

for test_case in test_cases:
  string, expected_vowels, expected_consonants = test_case
  vowels, consonants = count_vowels_and_consonants(string)
  if vowels == expected_vowels and consonants == expected_consonants:
    print('Test pass')
  else:
    print('Test fail')



In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = "count a number of leaf nodes in a given binary tree",
                 debug=False)

The generated code isn't working, asking the model to fix it
Fixing Try......  1
test fail
The generated code is running without any error, after fixing the issues: 



def count_leaf_nodes(root):
    if root is None:
        return 0
    elif root.left is None and root.right is None:
        return 1
    else:
        return count_leaf_nodes(root.left) + count_leaf_nodes(root.right)

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

if count_leaf_nodes(root) == 5:
    print("test pass")
else:
    print("test fail")



In [ ]:
problem = """
Sort an array in one swap whose two elements are swapped and rest are in sorted order \
                      for example: \
                      input: {1, 5, 3, 7, 9}
                      output: {1,3,5,7,9}
"""
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executer(code_chat_model, problem_statement = problem,
                 debug=False)

Test pass
Test fail
Test pass
Test fail
The generated code is running without any error, However it may still not pass some test cases: 



def sort_array_one_swap(arr):
    """
    Sorts an array in one swap whose two elements are swapped and rest are in sorted order.

    Args:
    arr: The array to be sorted.

    Returns:
    The sorted array.
    """

    # Find the two elements that are swapped.
    i = 0
    while i < len(arr) - 1 and arr[i] < arr[i + 1]:
        i += 1

    j = i + 1
    while j < len(arr) and arr[j] > arr[j - 1]:
        j += 1

    # Swap the two elements.
    arr[i], arr[j] = arr[j], arr[i]

    # Return the sorted array.
    return arr

# Test the function.
test_cases = [
    ([1, 5, 3, 7, 9], [1, 3, 5, 7, 9]),
    ([7, 9, 5, 3, 1], [1, 3, 5, 7, 9]),
    ([1, 3, 2, 4, 5], [1, 2, 3, 4, 5]),
    ([5, 4, 3, 2, 1], [1, 2, 3, 4, 5]),
]

for test_case in test_cases:
    arr, expected = test_case
    actual = sort_array_one_swap(arr)
    if actual == expected:
   

# Code Executor Class [Some Bug]

In [ ]:
import logging
from subprocess import run


class CodeExecutor:
    def __init__(self, chat_model: CodeChatModel, debug=True):
        """
        Initialize the CodeExecutor with a chat model.

        Parameters:
        - chat_model: The chat model for generating and fixing code.
        - debug (bool): If True, enable debug logging.
        """
        # self.chat_model = chat_model
        self.debug = debug
        self.logger = self._setup_logger()
        self.code_chat = chat_model.start_chat(temperature = 0,
                                       max_output_tokens=2048)

    def _setup_logger(self):
        logger = logging.getLogger(__name__)
        if self.debug:
            logging.basicConfig(level=logging.DEBUG)
        else:
            logging.basicConfig(level=logging.INFO)
        return logger

    def generate_code(self, problem_statement: str) -> str:
        """
        Generate Python code based on the provided problem statement.

        Parameters:
        - problem_statement (str): The problem statement for the code.

        Returns:
        str: Generated Python code.
        """
        step_1 = self.code_chat.send_message(
            f"Please help write a Python function {problem_statement}"
        )
        step_2 = self.code_chat.send_message(
            """
            Modify the provided Python function so that it can be executed in the Python kernel.
            Ensure the function includes necessary function calls and test cases.
            - Do not use the unittest library or create Python classes.
            - Make sure to add all the import statements at the top of the function.
            - Do not add "if __name__ == "__main__": to call the function. Just do simple function calling.
            - If the test passes, print "test pass".
            - If the test fails, print "test fail".
        """
        )

        self.logger.debug("1st generation:\n%s", step_2.text)

        code_to_execute = step_2.text.split("```")[1].split("python")[1]

        self.logger.debug("1st run code:\n%s", code_to_execute)

        return code_to_execute

    def execute_code(self, code: str) -> str:
        """
        Execute the provided Python code and return the execution result.

        Parameters:
        - code (str): Python code to be executed.

        Returns:
        str: Execution result, either "Success" or an error message.
        """
        try:
            exec(code)
            result = "Success"
        except Exception as e:
            error_message = str(e)
            self.logger.exception(
                "Error during code execution:\n%s", error_message
            )
            result = f"Error: {error_message}"
        return result

    def run_generated_code(self, problem_statement: str) -> None:
        """
        Generate and execute Python code based on the provided problem statement.

        Parameters:
        - problem_statement (str): The problem statement for the code.
        """
        generated_code = self.generate_code(problem_statement)
        execution_result = self.execute_code(generated_code)

        self.logger.debug("Code execution result:\n%s", execution_result)

        if execution_result == "Success":
            print(
                """The generated code is running without any error. However,
                   it may or maynot pass some test cases. Check the above prints to check.
                   Code: \n\n
                """
            )
            print(generated_code)
        else:
            self.logger.warning(
                "The generated code isn't working; asking the model to fix it"
            )
            print(
                "The generated code isn't working; asking the model to fix it"
            )
            run_flag = True
            max_iterations = 4
            i = 0

            while run_flag and i < max_iterations:
                self.logger.debug("Fixing Try...... %d", i + 1)
                print("Fixing Try...... %d", i + 1)
                step_3 = self.code_chat.send_message(
                    f"""The code throws an error: {execution_result}.
                        - Re-write the code so that it can run on the Python kernel.
                        - If the error is due to library installation \
                           add lines that can install the libraries with '!' like '!pip install pandas`.
                        """
                )
                code_to_execute = step_3.text.split("```")[1].split("python")[1]
                execution_result = self.execute_code(code_to_execute)

                if execution_result == "Success":
                    print(
                        "The generated code is running without any error after fixing the issues:\n\n"
                    )
                    print(code_to_execute)
                    run_flag = False
                else:
                    run_flag = True
                    i += 1
                    if i == max_iterations:
                        self.logger.error(
                            """There's something wrong with the code; the model is unable to fix it in 4 iterations.
                              You can run with debug=True and see for yourself.
                              Maybe re-write the prompt and ask again. Thank You!
                              """
                        )


In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executor = CodeExecutor(code_chat_model, debug=True)
problem_statement = """to print the first non-repeated character from a string.
                      take example of 'Mississippi' and 'hello' as example.
                    """
# Generate and run the code
code_executor.run_generated_code(problem_statement)

2023-10-07 09:47:08,352 - DEBUG - 1st generation:
 ```python
import collections

def first_non_repeating_char(str1):
    char_order = []
    char_count = collections.Counter(str1)

    for c in str1:
        if char_count[c] == 1:
            return c

    return -1

# Example 1:
str1 = "Mississippi"
result = first_non_repeating_char(str1)
if result == 's':
    print("test pass")
else:
    print("test fail")

# Example 2:
str2 = "hello"
result = first_non_repeating_char(str2)
if result == 'h':
    print("test pass")
else:
    print("test fail")
```

Output:

```
test pass
test pass
```
2023-10-07 09:47:08,352 - DEBUG - 1st generation:
 ```python
import collections

def first_non_repeating_char(str1):
    char_order = []
    char_count = collections.Counter(str1)

    for c in str1:
        if char_count[c] == 1:
            return c

    return -1

# Example 1:
str1 = "Mississippi"
result = first_non_repeating_char(str1)
if result == 's':
    print("test pass")
else:
    print("test fa

The generated code isn't working; asking the model to fix it
Fixing Try...... %d 1


2023-10-07 09:47:11,613 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:11,613 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:11,613 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
ERROR:__main__:Error during code execution:
invalid syntax (<string>, line 2)
Traceback (mo

Fixing Try...... %d 2


2023-10-07 09:47:14,860 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:14,860 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:14,860 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
ERROR:__main__:Error during code execution:
invalid syntax (<string>, line 2)
Traceback (mo

Fixing Try...... %d 3


2023-10-07 09:47:18,092 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:18,092 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:18,092 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
ERROR:__main__:Error during code execution:
invalid syntax (<string>, line 2)
Traceback (mo

Fixing Try...... %d 4


2023-10-07 09:47:21,290 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:21,290 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
2023-10-07 09:47:21,290 - ERROR - Error during code execution:
invalid syntax (<string>, line 2)
Traceback (most recent call last):
  File "<ipython-input-11-73d0f29c6bb8>", line 72, in execute_code
    exec(code)
  File "<string>", line 2
    !pip install collections
    ^
SyntaxError: invalid syntax
ERROR:__main__:Error during code execution:
invalid syntax (<string>, line 2)
Traceback (mo

In [ ]:
exec(
"""
import collections

def first_non_repeating_char(str1):
    char_order = []
    char_count = collections.Counter(str1)

    for c in str1:
        if char_count[c] == 1:
            return c

    return -1

# Example 1:
str1 = "Mississippi"
result = first_non_repeating_char(str1)
if result == 's':
    print("test pass")
else:
    print("test fail")

# Example 2:
str2 = "hello"
result = first_non_repeating_char(str2)
if result == 'h':
    print("test pass")
else:
    print("test fail")
"""
)

test fail
test pass


In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executor = CodeExecutor(code_chat_model, debug=False)
problem_statement = """that can extract ipv4 addresses from the each line in the log file.
                        input:
                        03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0
                        output:
                        129.1.1.1
                        \n\n
                    """
# Generate and run the code
code_executor.run_generated_code(problem_statement)

test pass
The generated code is running without any error. However, it may still not pass some test cases:



import re

def extract_ipv4_addresses(line):
  """Extracts IPv4 addresses from a line of text."""

  # Compile a regular expression to match IPv4 addresses.
  ipv4_regex = re.compile(r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b")

  # Find all IPv4 addresses in the line of text.
  ipv4_addresses = ipv4_regex.findall(line)

  # Return the list of IPv4 addresses.
  return ipv4_addresses


# Test the function.
line = "03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0"
expected_ipv4_addresses = ["129.1.1.1"]
ipv4_addresses = extract_ipv4_addresses(line)
if ipv4_addresses == expected_ipv4_addresses:
  print("test pass")
else:
  print("test fail")



In [ ]:
problem_statement = """
that defines a class called Restaurant with three attributes: name, address, and cuisine.
Then, write a function called find_matching_restaurants() that takes a list of Restaurant objects and a search query \
as input and returns a list of Restaurant objects that match the search query.
The search query can be a substring of the restaurant name, address, or cuisine type.
here's an example of using the following function:
restaurants = [
              Restaurant("The Grill", "123 Main Street", cuisine="American"),
              Restaurant("Thai Paradise", "456 Elm Street", cuisine="Thai"),
              Restaurant("Little Italy", "789 Pine Street", cuisine="Italian"),
              ]
search_query = "Italian"
output: 'Little Italy'

"""
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
code_executor = CodeExecutor(code_chat_model, debug=False)
code_executor.run_generated_code(problem_statement)

test pass
test pass
test pass
test fail
test pass
The generated code is running without any error. However, it may still not pass some test cases:



import re

def is_valid_email(email):
    # check for valid email format
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", email):
        return True
    else:
        return False

# Test cases
test_cases = ["john@example.com", "example@example.com", "example123@example.co.in", "example@example", "example@example.c"]
for email in test_cases:
    if is_valid_email(email):
        print("test pass")
    else:
        print("test fail")



# Test Driven Development

In [ ]:
%%writefile config.yaml
database:
    host: localhost
    port: 5432
    username: postgres
    password: secret
logging:
    level: INFO
    file: /var/log/myapp.log


Writing config.yaml


In [ ]:
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Print the database host
print(config["database"]["host"])

# Print the logging level
print(config["logging"]["level"])

localhost
INFO


In [ ]:
from subprocess import run


def execute_code(code):
    try:
        # exec(code)
        result = "Success"
        # Create a dictionary to store the local variables
        local_vars = {}

        # Execute the code within the local_vars dictionary
        exec(code, globals(), local_vars)

        # Access the result from the local_vars dictionary
        result = local_vars
    except Exception as e:
        result = f"Error: {str(e)}"
    return result


def check_tdd_flag(execution_result_dict):
  # it returns True if any test cases are failing.
    for key, value in execution_result_dict.items():
        if "fail" in value:
            return True

def filter_test_that_fails(execution_result_dict):
  # it returns True if any test cases are failing.
    pending_test_cases = {}
    pending_test_example = []
    for key, value in execution_result_dict['test_results'].items():
        if "fail" in value:
            pending_test_cases[key] = value
    for eachValue in execution_result_dict['test_cases']:
      if eachValue[0] in pending_test_cases.keys():
        pending_test_example.append(eachValue)
    return pending_test_cases, pending_test_example

In [ ]:
#phase2: tries to fix the phase 1 code where it is failing on some test cases. It will try to modify the code to address the failing test cases

def code_executer_phase_2_tdd(
    code_chat_obj,
    generated_code="",
    execution_result_dict_before_tdd = {},
    debug=False,
    max_iter_tdd=2,
    modified_test_cases="",
    extra_instructions="",
):
    #pass 1
    print("execution_result_dict_before_tdd", execution_result_dict_before_tdd)
    pending_test_cases, pending_test_example = filter_test_that_fails(execution_result_dict_before_tdd)
    print("First pass test case and example :\n", pending_test_cases, pending_test_example)
    step_1 = code_chat_obj.send_message(
        f"""
          fix the bug that is causing the code to fail on the following test case:
          test_cases: {pending_test_example}
          the current test results are as follows: {pending_test_cases}.
          if the test cases has 'Invalid Input' add exception ahndling as they might be wrong or unexpected inputs.
          The goal is to make sure the current code solved all the test cases and passes all the result.
          code: {generated_code}
          """,
    )
    code_to_execute = step_1.text.split("```")[1].split("python")[1]
    execution_result_phase2 = execute_code(code_to_execute)

    #Other Iter
    tdd_i = 0
    tdd_iter_flag = False
    if check_tdd_flag(execution_result_phase2["test_results"]):
      tdd_iter_flag = True
      print("Codey is failing to fix the test cases issues....")
    while(tdd_iter_flag and tdd_i < max_iter_tdd):
      print("Fixing Try...... ", tdd_i + 1)
      pending_test_cases, pending_test_example = filter_test_that_fails(execution_result_phase2)
      print("inside iteration test case and example :\n", pending_test_cases, pending_test_example)
      step_1 = code_chat_obj.send_message(
        f"""
          the bugs are really critical and are unable to pass the test.
          fix the bug that is causing the code to fail on the following test case even if it requires adding extra exception handling:
          test_cases: {pending_test_example}
          the current test results are as follows: {pending_test_cases}.
          if the test cases has 'Invalid Input' add exception ahndling as they might be wrong or unexpected inputs.
          The goal is to make sure the current code solved all the test cases and passes all the result.
          code: {generated_code}
          """,
      )
      code_to_execute = step_1.text.split("```")[1].split("python")[1]
      execution_result = execute_code(code_to_execute)

      if check_tdd_flag(execution_result["test_results"]):
        tdd_iter_flag = True
        tdd_i += 1
        if tdd_i == max_iter_tdd:
          print(f"""Sorry Codey is unable to fix failing test cases in {max_iter_tdd} iterations.
                    Here are open test cases which are still pending: {execution_result["test_results"]}
                    You can run with debug=True and see for yourself.
                    Maybe re-write the prompt and re-check your test cases and ask again.
                    Thank You! Codey tried its best :)
                """
          )
      else:
        tdd_iter_flag = False

    return code_to_execute, execution_result


In [ ]:
#Phase1 - Base generation that make sures that there is a running code with proper coding standards.
def code_executer_phase_1(
    chat_model, problem_statement="", debug=True,
    max_iter_tdd=2,
    extra_instructions=""
):
    run_flag = False
    code_chat = chat_model.start_chat(temperature=0, max_output_tokens=2048)
    step_1 = code_chat.send_message(
        f"""
          Please help write a python function {problem_statement}
          """,
    )
    step_2 = code_chat.send_message(
        """
         Modify the previous code so that it can be executed in the Python kernel with following condition:
            - Make sure to add all the import statements at the top of the function.
            - Add exception handling for input sanitization.
            - If it’s using any libraries which are not default, add !pip install for that.
            - Add docstrings and comments wherever possible.
            - Add type hints wherever possible
            - Format the code to pep8 formatting
        """
    )
    step_3 = code_chat.send_message(
        """
        Modify the previous code so that it can be executed in the Python kernel with following condition:
            - Do not use unittest function and if __name__ == "__main__"
            - Ensure the function includes necessary function calls.
            - If user has specified test cases, add them as 'test_cases' \
                as list of tuples `[(input, output), (input, output)]`. Do not use any other name or convention.
            - If user has not added any test cases then geenrate based on the problem statement.
            - If the test passes, print "test pass". Do not add any other string for test pass.
            - If the test fails, print "test fail". Do not add any other string for test fail.
            - Save the test results in a test_results `dictionary` that includes the test cases as key and their results as values \
            `{'input': 'test pass', 'input': 'test fail'}` . Do not use any other name or convention.
      """
    )

    if extra_instructions:
        step_extra_isntruction = code_chat.send_message(
            f"""
        {extra_instructions}
        """
        )
        split_string_object = step_extra_isntruction
    else:
        split_string_object = step_3

    code_to_execute = split_string_object.text.split("```")[1].split("python")[1]
    execution_result = execute_code(code_to_execute)

    if "Error" in execution_result:
        print("The generated code isn't working, asking the model to fix it")
        run_flag = True
        i = 0
        while run_flag and i < 2:
            print("Fixing Try...... ", i + 1)
            step_test = code_chat.send_message(
                f"""The code throws an error: {execution_result}.
              - Re-write the code so that it can run on the Python kernel.
              - If the error is due to library installation \
                  add lines that can install the libraries with '!' like '!pip install pandas`.
              """
            )
            code_to_execute = step_test.text.split("```")[1].split("python")[1]
            execution_result = execute_code(code_to_execute)
            if "Error" not in execution_result:
                print(
                    "The generated code is running without any error, after fixing the issues: \n\n"
                )
                # print(code_to_execute)
                run_flag = False
            else:
                run_flag = True
                i += 1
                if i == 2:
                    print(
                        """There's something wrong with the code, model is unable to fix in 4 iterations.
                  You can run with debug=True and see for yourself.
                  Maybe re-write the prompt and ask again. Thank You!
                  """
                    )
    else:
        print(
            "First pass code is working without any issues in Python Kernel. Good Job Codey......Now checking test cases clause."
        )
    print("execution_result before TDD: \n", execution_result)
    print("TDD FLAG: ", check_tdd_flag(execution_result["test_results"]))
    if check_tdd_flag(execution_result["test_results"]):
        print(
            "Some of the test has been failing. Make sure you have given correct test cases......"
        )
        code_chat_tdd = code_chat = chat_model.start_chat(
            temperature=0, max_output_tokens=2048
        )
        (
            code_to_execute_after_tdd,
            execution_result_after_tdd,
        ) = code_executer_phase_2_tdd(
            code_chat_tdd,
            generated_code=code_to_execute,
            execution_result_dict_before_tdd = execution_result,
            max_iter_tdd=max_iter_tdd,
            debug=False,
        )
        tdd_post_flag = check_tdd_flag(execution_result_after_tdd["test_results"])
        if tdd_post_flag:
            print(
                "The code is still not able to pass all the tests. Check and modify your test."
            )
        else:
            print("All test are passing, you have the complete santized code.")
    else:
        code_to_execute_after_tdd, execution_result_after_tdd = None, None
        print("No need for TDD Phase, since all test are passed at Phase 1")

    return (
        code_to_execute,
        execution_result,
        code_to_execute_after_tdd,
        execution_result_after_tdd,
    )

# import pandas as pd
# if execution_result_after_tdd and code_to_execute_after_tdd :
#   print("Here's the final code: \n ", code_to_execute_after_tdd)
#   pd.DataFrame(execution_result_after_tdd['test_results'],index=[0]).T
# else:
#   print("Here's the final code: \n ", code_to_execute_before_tdd)
#   pd.DataFrame(execution_result_before_tdd['test_results'],index=[0]).T

In [ ]:
user_test_cases = """
input: "Mississippi"
output: "M"
input: "hello"
output: "h"
input: "GeeksforGeeks"
output: "f"
input: "abcdabcd"
output: ""
input: ""
output: ""
input: "12234439916"
output: "6"
"""
problem_statement = f"""to find the first non-repeated character from a string.
                        A string can be both characters and numbers given as strings.
                        The input and output to this function will be like this:
                        {user_test_cases}
                      """
print ("Debug Statements ************************************************** \n\n\n")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
(code_to_execute_before_tdd,
 execution_result_before_tdd,
 code_to_execute_after_tdd,
 execution_result_after_tdd) = code_executer_phase_1(code_chat_model,
                                                     problem_statement = problem_statement,
                                                     max_iter_tdd = 2
                                                    )
print ("Debug Statements End************************************************** \n\n\n")

Debug Statements ************************************************** 



Mississippi: test pass
hello: test pass
GeeksforGeeks: test pass
abcdabcd: test pass
: test pass
12234439916: test pass
First pass code is working without any issues in Python Kernel. Good Job Codey......Now checking test cases clause.
execution_result before TDD: 
 {'collections': <module 'collections' from '/usr/lib/python3.10/collections/__init__.py'>, 'first_non_repeating_char': <function first_non_repeating_char at 0x792eb29cc9d0>, 'test_cases': [('Mississippi', 'M'), ('hello', 'h'), ('GeeksforGeeks', 'f'), ('abcdabcd', ''), ('', ''), ('12234439916', '6')], 'test_results': {'Mississippi': 'test pass', 'hello': 'test pass', 'GeeksforGeeks': 'test pass', 'abcdabcd': 'test pass', '': 'test pass', '12234439916': 'test pass'}, 'test_case': '12234439916', 'input': '12234439916', 'expected_output': '6', 'output': '6', 'result': 'test pass'}
TDD FLAG:  None
No need for TDD Phase, since all test are passed at Phase 1
D

In [ ]:
pd.DataFrame(execution_result_before_tdd['test_results'],index=[0]).T

,0
Mississippi,test pass
hello,test pass
GeeksforGeeks,test pass
abcdabcd,test pass
,test pass
12234439916,test pass


In [ ]:
# import pandas as pd
# if execution_result_after_tdd and code_to_execute_after_tdd :
#   print("Here's the final code: \n ", code_to_execute_after_tdd)
#   pd.DataFrame(execution_result_after_tdd['test_results'],index=[0]).T
# else:
#   print("Here's the final code: \n ", code_to_execute_before_tdd)
#   pd.DataFrame(execution_result_before_tdd['test_results'],index=[0]).T

In [ ]:
user_test_cases = """
input:"/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0"
output:["129.1.1.1"]
input:"03/22 08:53:53 TRACE  :......router_forward_getOI:         source address:   9.67.116.98"
output: [“9.67.116.98”]
input: “03/22 08:53:53 INFO   :......rsvp_flow_stateMachine: state RESVED, event T1OUT”
output: []
input: "03/22 08:54:09 TRACE  :......router_forward_getOI:         out inf:   9.67.116.98"
output:[“9.67.116.98”]
input: "03/22 08:54:22 INFO   :.......rsvp_parse_objects: obj RSVP_HOP hop=9.67.116.99, lih=0"
output:[“9.67.116.99"]
Input: “03/22 08:54:22 TRACE  :......rsvp_event_mapSession: Session=9.67.116.99:1047:6 exists”
output: ["9.67.116.99"]
Input: “03/22 08:54:24 TRACE  :.......mailslot_send: sending to (9.67.116.99:0)”
output: [“9.67.116.99”]
input: “03/22 08:54:35 INFO   :..........qosmgr_response: src-9.67.116.98:8000 dst-9.67.116.99:1047 proto-6”
output: [“9.67.116.98”, “9.67.116.99”]
input: “03/22 08:52:52 INFO   :...........traffic_reader: Reservation req successful[session=9.67.116.99:1047:6,
source=9.67.116.98:8000, qoshd=8b671d0]”
output: [“9.67.116.99”, “9.67.116.98”]
"""
problem_statement = f"""that can extract single or multiple ipv4 addresses as list from the each line in the log file.
                        The input and output to this function will be like this:
                        {user_test_cases}
                      """
print ("Debug Statements ************************************************** \n\n\n")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
(code_to_execute_before_tdd,
 execution_result_before_tdd,
 code_to_execute_after_tdd,
 execution_result_after_tdd) = code_executer_phase_1(code_chat_model,
                                                     problem_statement = problem_statement,
                                                     max_iter_tdd = 2
                                                    )
print ("Debug Statements End************************************************** \n\n\n")

Debug Statements ************************************************** 



/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0: test pass
03/22 08:53:53 TRACE  :......router_forward_getOI:         source address:   9.67.116.98: test pass
03/22 08:53:53 INFO   :......rsvp_flow_stateMachine: state RESVED, event T1OUT: test pass
03/22 08:54:09 TRACE  :......router_forward_getOI:         out inf:   9.67.116.98: test pass
03/22 08:54:22 INFO   :.......rsvp_parse_objects: obj RSVP_HOP hop=9.67.116.99, lih=0: test pass
03/22 08:54:22 TRACE  :......rsvp_event_mapSession: Session=9.67.116.99:1047:6 exists: test pass
03/22 08:54:24 TRACE  :.......mailslot_send: sending to (9.67.116.99:0): test pass
03/22 08:54:35 INFO   :..........qosmgr_response: src-9.67.116.98:8000 dst-9.67.116.99:1047 proto-6: test pass
03/22 08:52:52 INFO   :...........traffic_reader: Reservation req successful[session=9.67.116.99:1047:6,
source=9.67.116.98:8000, qoshd=8b671d0

In [ ]:
pd.DataFrame(execution_result_before_tdd['test_results'],index=[0]).T

,0
"/22 08:51:06 INFO :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0",test pass
03/22 08:53:53 TRACE :......router_forward_getOI: source address: 9.67.116.98,test pass
"03/22 08:53:53 INFO :......rsvp_flow_stateMachine: state RESVED, event T1OUT",test pass
03/22 08:54:09 TRACE :......router_forward_getOI: out inf: 9.67.116.98,test pass
"03/22 08:54:22 INFO :.......rsvp_parse_objects: obj RSVP_HOP hop=9.67.116.99, lih=0",test pass
03/22 08:54:22 TRACE :......rsvp_event_mapSession: Session=9.67.116.99:1047:6 exists,test pass
03/22 08:54:24 TRACE :.......mailslot_send: sending to (9.67.116.99:0),test pass
03/22 08:54:35 INFO :..........qosmgr_response: src-9.67.116.98:8000 dst-9.67.116.99:1047 proto-6,test pass
"03/22 08:52:52 INFO :...........traffic_reader: Reservation req successful[session=9.67.116.99:1047:6,\nsource=9.67.116.98:8000, qoshd=8b671d0]",test pass


In [ ]:
user_test_cases = """
input: "aeiou"
output: [5,0]
input: "bcdfghjklmnpqrstvwxyz"
output: [0,26]
input: "This is a simple sentence."
output: [5,13]
input: "Hello world!"
output: [3,7]
input: ""
output: [0,0]
input: " "
output: [0,0]
input: "~!@#$%^&*()_+-=[]{};:,<.>/?\|"
output: [0,27]
input: "1234567890"
output: [0,10]
input: "aeiouAEIOU"
output: [10,0]
input: "bcdfghjklmnpqrstvwxyzBCDFGHIJKLMNPQRSTVWXYZ"
output: [0,52]
input: "This is a simple sentence with mixed cases."
output: [10,13]
input: "Hello world! with punctuation."
output: [3,7]
"""
problem_statement = f"""count a number of vowels and consonants in a given string.
                        The input and output to this function will be like this:
                        {user_test_cases}
                      """
print ("Debug Statements ************************************************** \n\n\n")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
(code_to_execute_before_tdd,
 execution_result_before_tdd,
 code_to_execute_after_tdd,
 execution_result_after_tdd) = code_executer_phase_1(code_chat_model,
                                                     problem_statement = problem_statement,
                                                     max_iter_tdd = 2
                                                    )
print ("Debug Statements End************************************************** \n\n\n")

Debug Statements ************************************************** 



aeiou: test pass
bcdfghjklmnpqrstvwxyz: test fail
This is a simple sentence.: test fail
Hello world!: test pass
: test pass
 : test pass
~!@#$%^&*()_+-=[]{};:,<.>/?\|: test fail
1234567890: test fail
aeiouAEIOU: test pass
bcdfghjklmnpqrstvwxyzBCDFGHIJKLMNPQRSTVWXYZ: test fail
This is a simple sentence with mixed cases.: test fail
Hello world! with punctuation.: test fail
First pass code is working without any issues in Python Kernel. Good Job Codey......Now checking test cases clause.
execution_result before TDD: 
 {'re': <module 're' from '/usr/lib/python3.10/re.py'>, 'count_vowels_and_consonants': <function count_vowels_and_consonants at 0x792eb29a4700>, 'test_cases': [('aeiou', [5, 0]), ('bcdfghjklmnpqrstvwxyz', [0, 26]), ('This is a simple sentence.', [5, 13]), ('Hello world!', [3, 7]), ('', [0, 0]), (' ', [0, 0]), ('~!@#$%^&*()_+-=[]{};:,<.>/?\\|', [0, 27]), ('1234567890', [0, 10]), ('aeiouAEIOU', [10, 0]), ('

In [ ]:
pd.DataFrame(execution_result_before_tdd['test_results'],index=[0]).T

,0
aeiou,test pass
bcdfghjklmnpqrstvwxyz,test fail
This is a simple sentence.,test fail
Hello world!,test pass
,test pass
,test pass
"~!@#$%^&*()_+-=[]{};:,<.>/?\|",test fail
1234567890,test fail
aeiouAEIOU,test pass
bcdfghjklmnpqrstvwxyzBCDFGHIJKLMNPQRSTVWXYZ,test fail


In [ ]:
print(code_to_execute_before_tdd)


import re

def count_vowels_and_consonants(string: str) -> list:
  """Counts the number of vowels and consonants in a given string.

  Args:
    string: The string to count the vowels and consonants in.

  Returns:
    A list containing the number of vowels and consonants in the string.
  """

  try:
    # Check if the input is a string
    if not isinstance(string, str):
      raise ValueError("Input must be a string.")

    # Sanitize the input string
    string = re.sub(r"[^a-zA-Z]", "", string)

    # Count the number of vowels and consonants
    vowels = "aeiouAEIOU"
    consonants = "bcdfghjklmnpqrstvwxyzBCDFGHIJKLMNPQRSTVWXYZ"
    vowels_count = 0
    consonants_count = 0
    for char in string:
      if char in vowels:
        vowels_count += 1
      elif char in consonants:
        consonants_count += 1

    return [vowels_count, consonants_count]
  except ValueError as e:
    print(e)
    return [0, 0]


# Test cases
test_cases = [
    ("aeiou", [5, 0]),
    ("bcdfghjklmnpqr

In [ ]:
Sort an array in one swap whose two elements are swapped and rest are in sorted order \
                      for example: \
                      input: {1, 5, 3, 7, 9}
                      output: {1,3,5,7,9}

In [ ]:
user_test_cases = """
#normal_case
input: [1, 5, 3, 7, 9]
output: [1,3,5,7,9]
#edge_case
input: []
output: []
#edge_case
input: [3, 5, 3, 3, 3]
output: [3, 3, 3, 3, 5]
#edge_case
input: [1, 5, 5, 7, 9]
output: [1, 5, 5, 7, 9]
#normal_case
input: [1, 3, 5, 9, 7]
output: [1, 3, 5, 7, 9]
#normal_case
input: [3, 3, 3, 3, 3]
output: [3, 3, 3, 3, 3]
#normal_case
input: [5, 3]
output: [3, 5]
"""
problem_statement = f"""Sort an array in one swap whose two elements are swapped and rest are in sorted order.
                        The input and output to this function will be like this:
                        {user_test_cases}
                      """
print ("Debug Statements ************************************************** \n\n\n")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison")
(code_to_execute_before_tdd,
 execution_result_before_tdd,
 code_to_execute_after_tdd,
 execution_result_after_tdd) = code_executer_phase_1(code_chat_model,
                                                     problem_statement = problem_statement,
                                                     max_iter_tdd = 2
                                                    )
print ("Debug Statements End************************************************** \n\n\n")

Debug Statements ************************************************** 



The generated code isn't working, asking the model to fix it
Fixing Try......  1
[1, 3, 5, 7, 9]: test pass
[]: test pass
[3, 3, 5, 3, 3]: test fail
[1, 5, 5, 7, 9]: test pass
[3, 3, 3, 3, 3]: test pass
[3, 5]: test pass
The generated code is running without any error, after fixing the issues: 


execution_result before TDD: 
 {'sys': <module 'sys' (built-in)>, 'List': typing.List, 'sort_array_in_one_swap': <function sort_array_in_one_swap at 0x7f92301b6320>, 'test_cases': [([1, 3, 5, 7, 9], [1, 3, 5, 7, 9]), ([], []), ([3, 3, 5, 3, 3], [3, 3, 3, 3, 5]), ([1, 5, 5, 7, 9], [1, 5, 5, 7, 9]), ([1, 3, 5, 7, 9], [1, 3, 5, 7, 9]), ([3, 3, 3, 3, 3], [3, 3, 3, 3, 3]), ([3, 5], [3, 5])], 'test_results': {'[1, 3, 5, 7, 9]': 'test pass', '[]': 'test pass', '[3, 3, 5, 3, 3]': 'test fail', '[1, 5, 5, 7, 9]': 'test pass', '[3, 3, 3, 3, 3]': 'test pass', '[3, 5]': 'test pass'}, 'test_case': '[3, 5]', 'input': [3, 5], 'expected_ou

TypeError: ignored

In [ ]:
user_input:
    problem_statement:
    sample_test_cases:
      #format [minimum three of each]
          #normal_case
          input:
          output:
          #edge_case
          input:
          output:

Phase_1:
    - Modify the prompt.


# AutoCodey

In [ ]:
from vertexai.language_models import TextGenerationModel
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
type(generation_model)

vertexai.language_models.TextGenerationModel

In [ ]:
prompt = "What is a large language model?"
response = generation_model.predict(prompt=prompt)
print(response.text)

In [ ]:
prompt_reengineering_log ={
 'Goal': "a",

  'Task': "b" ,

  'Persona': "c"

}

prompt_reengineering_log

{'Goal': 'a', 'Task': 'b', 'Persona': 'c'}

In [ ]:
  # if add_approach and add_persona:
  #   final_prompt_response = f"""Persona: {prompt_reengineering_log['Stage3_Prompt']} \n
  #                               Goal: {prompt_reengineering_log['Stage2_Prompt']}
  #                           """
  # elif add_approach and not add_persona:
  #   final_prompt_response = f"""Goal: {prompt_reengineering_log['Stage2_Prompt']}
  #                           """
  # elif add_persona and not add_approach:
  #   final_prompt_response = f"""Persona : {prompt_reengineering_log['Stage3_Prompt']} \n
  #                               Goal: {prompt_reengineering_log['Stage1_Prompt']} \n
  #                               """
  # else:
  #   final_prompt_response = f"""Goal: {prompt_reengineering_log['Stage2_Prompt']}
  #                           """

  # prompt_reengineering_log['Final_Prompt'] = final_prompt_response

In [ ]:
def prompt_reengineering(generation_model, user_prompt="",
                         temperature = 0,
                         max_output_tokens = 1024,
                         add_approach = True,
                         add_persona = True,

                         ):

  prompt_reengineering_log = {}
  prompt_reengineering_log['Base_Prompt'] = user_prompt

  #Step1: Fix the grammar and spelling mistakes. Mandadotry Step.
  step_prompt_stage1 = f"""
                Rewrite the following statement that fixes grammar and spelling mistakes.
                Do not add python code or example.
                {user_prompt}
                """
  response_fixed_stage1 = generation_model.predict(prompt=step_prompt_stage1,
                                                   temperature=temperature,
                                                   max_output_tokens = max_output_tokens).text

  prompt_reengineering_log['Stage1_Prompt'] = response_fixed_stage1
  print("Stage 1:",response_fixed_stage1)

  if add_approach:
    #Step2:Adding extra information and approach:
    step_prompt_stage2 = f"""
                  can you rewrite 'goal:' this like you have to explain this to a recent college graduate who \
                  has started learning programming.
                  - Add a simple english approach to solving this problem.
                  - Do not add any simple or sample python function.
                  - The goal should start with this line "Write a python program"
                  - The final result should not have any code blocks, just plain english.
                  goal:
                  {prompt_reengineering_log['Stage1_Prompt']}
                  """
    response_fixed_stage2 = generation_model.predict(prompt=step_prompt_stage2,
                                                    temperature=temperature,
                                                    max_output_tokens = max_output_tokens).text

    prompt_reengineering_log['Stage2_Prompt'] = response_fixed_stage2
    print("Stage 2:",response_fixed_stage2)

  if add_persona:
    #Step3: Adding persona to the prompt
    step_prompt_stage3 = f"""
                  Add persona to the following 'goal' such that LLMs can be preempted to provide more \
                  contextual output. Persona should include:\
                    - Do not add any name, company name, race or ethenicity.
                    - Their role and position in the company.
                    - Why solving the goal.
                    - What are the end goals.
                    - How would solving the rpblem help them.
                  It should be a paragrapgh and not bullets.
                  Do not add python code or example.
                  goal:
                  {prompt_reengineering_log['Stage2_Prompt'] if add_approach else prompt_reengineering_log['Stage1_Prompt'] }
                  """
    response_fixed_stage3 = generation_model.predict(prompt=step_prompt_stage3,
                                                    temperature=temperature,
                                                    max_output_tokens = max_output_tokens).text

    prompt_reengineering_log['Stage3_Prompt'] = response_fixed_stage3
    print("Stage 3:",response_fixed_stage3)


  return prompt_reengineering_log

In [ ]:
%%time
generation_model = TextGenerationModel.from_pretrained("text-bison")
user_prompt = "Please help write a python function to calculate the moving average of a stock price over a given period of time."
prompt_reengineering_log = prompt_reengineering(generation_model,
                     user_prompt=user_prompt,
                     max_output_tokens = 2048,
                     add_approach= False,
                     add_persona= False
                      )
prompt_reengineering_log

Stage 1:  Can you please write a python function to calculate the moving average of a stock price over a given period of time? 
CPU times: user 39.4 ms, sys: 4.43 ms, total: 43.8 ms
Wall time: 2.09 s


{'Base_Prompt': 'Please help write a python function to calculate the moving average of a stock price over a given period of time.',
 'Stage1_Prompt': ' Can you please write a python function to calculate the moving average of a stock price over a given period of time? '}

In [ ]:
prompt_reengineering_log

{'Base_Prompt': 'Please help write a python function to calculate the moving average of a stock price over a given period of time.',
 'Stage1_Prompt': ' Can you please write a python function to calculate the moving average of a stock price over a given period of time? ',
 'Stage2_Prompt': ' Write a python program that will take in a list of stock prices and a period of time, and then calculate and return the moving average of the stock price over that period of time. \n\nTo calculate the moving average, you will need to: \n1. Add up the stock prices for the given period of time. \n2. Divide the sum by the number of stock prices in the period. \n\nThe moving average is a simple way to smooth out the stock price data and identify trends. It can be used to help investors make decisions about when to buy or sell a stock.',
 'Stage3_Prompt': " As a data analyst at a large investment firm, I am tasked with developing a Python program that will take in a list of stock prices and a period of 

In [ ]:
%%time
generation_model = TextGenerationModel.from_pretrained("text-bison")
user_prompt = "Please help write a python function to calculate the moving average of a stock price over a given period of time."
prompt_reengineering_log, final_prompt = prompt_reengineering(generation_model,
                     user_prompt=user_prompt,
                     max_output_tokens = 2048
                                  )
print("\n\n Final ReEngineered Prompt: \n",final_prompt)

Stage 1:  Can you please write a python function to calculate the moving average of a stock price over a given period of time? 
Stage 2:  Write a python program that will take in a list of stock prices and a period of time, and then calculate and return the moving average of the stock price over that period of time. 

To calculate the moving average, you will need to: 
1. Add up the stock prices for the given period of time. 
2. Divide the sum by the number of stock prices in the period. 

The moving average is a simple way to smooth out the stock price data and identify trends. It can be used to help investors make decisions about when to buy or sell a stock.
Stage 3:  Alex is a data analyst at a large investment firm. He is responsible for developing and implementing quantitative models to help the firm make investment decisions. Alex is interested in using moving averages to identify trends in stock prices. He believes that by using moving averages, he can help the firm make more in

In [ ]:
%%time
generation_model = TextGenerationModel.from_pretrained("text-bison")
user_prompt = "Please help write a python fnctn to Sort an ary in one swap whose two elements are swapped and rest are in sorted order"
prompt_reengineering_log, final_prompt = prompt_reengineering(generation_model,
                     user_prompt=user_prompt,
                     max_output_tokens = 2048
                                  )
print("\n\n Final ReEngineered Prompt: \n",final_prompt)

Stage 1:  Please help write a python function to sort an array in one swap whose two elements are swapped and rest are in sorted order.
Stage 2:  Write a python program that takes in an array of integers as input. 
                 The array contains all the elements in sorted order except for two elements which are swapped. 
                 The goal of the program is to sort the array in one swap.

                 To achieve this, we can use the following approach: 
                 1. Find the two elements that are swapped. 
                 2. Swap the two elements. 
                 3. Return the sorted array. 

                 Here is a simple English approach to solving this problem: 
                 1. Iterate through the array and find the two elements that are swapped. 
                 2. Store the indices of the two elements in variables. 
                 3. Swap the two elements using the indices. 
                 4. Return the sorted array.
Stage 3:  A software engin

In [ ]:
%%time
generation_model = TextGenerationModel.from_pretrained("text-bison")
user_prompt = """Please help write a python fnctn that can extract single or multiple ipv4 addresses as list from the each line in the log file.
               The input will be just single lines and not a log file.
               """
prompt_reengineering_log, final_prompt = prompt_reengineering(generation_model,
                     user_prompt=user_prompt,
                     max_output_tokens = 2048
                                  )
print("\n\n Final ReEngineered Prompt: \n",final_prompt)

Stage 1:  Can you please help me write a python function that can extract a single or multiple IPv4 addresses as a list from each line in a log file? The input will be just single lines and not a log file.
Stage 2:  Write a python program that does the following: 
                1. Takes in a single line of text as input. 
                2. Extracts all IPv4 addresses from the line of text. 
                3. Returns the IPv4 addresses as a list. 
                4. Repeats steps 1-3 for each line in the input text file. 
                5. Prints the list of IPv4 addresses to the console.
Stage 3:  A software engineer at a large tech company is tasked with developing a program that can extract IPv4 addresses from a given text file. 
The engineer is motivated by the need to improve the security of the company's network by identifying and tracking all devices connected to it. 
The engineer believes that by solving this problem, they will be able to better protect the company's data a

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

pd.DataFrame(prompt_reengineering_log,index=[0])

,Base_Prompt,Stage1_Prompt,Stage2_Prompt,Stage3_Prompt,Final_Prompt
0,Please help write a python fnctn that can extract single or multiple ipv4 addresses as list from the each line in the log file.\n The input will be just single lines and not a log file. \n,Can you please help me write a python function that can extract a single or multiple IPv4 addresses as a list from each line in a log file? The input will be just single lines and not a log file.,Write a python program that does the following: \n 1. Takes in a single line of text as input. \n 2. Extracts all IPv4 addresses from the line of text. \n 3. Returns the IPv4 addresses as a list. \n 4. Repeats steps 1-3 for each line in the input text file. \n 5. Prints the list of IPv4 addresses to the console.,"A software engineer at a large tech company is tasked with developing a program that can extract IPv4 addresses from a given text file. \nThe engineer is motivated by the need to improve the security of the company's network by identifying and tracking all devices connected to it. \nThe engineer believes that by solving this problem, they will be able to better protect the company's data and systems from unauthorized access.","Persona: A software engineer at a large tech company is tasked with developing a program that can extract IPv4 addresses from a given text file. \nThe engineer is motivated by the need to improve the security of the company's network by identifying and tracking all devices connected to it. \nThe engineer believes that by solving this problem, they will be able to better protect the company's data and systems from unauthorized access. \n \n Goal: Write a python program that does the following: \n 1. Takes in a single line of text as input. \n 2. Extracts all IPv4 addresses from the line of text. \n 3. Returns the IPv4 addresses as a list. \n 4. Repeats steps 1-3 for each line in the input text file. \n 5. Prints the list of IPv4 addresses to the console.\n"
